In [1]:
!pip install rouge-score bert-score sacrebleu nltk



  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.6 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=3bffaf0c0f44c322249c601f2d22d0c074b290182f10850a66910fedfb8a62fb
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [2]:
!pip install --no-cache-dir numpy pyemd transformers jiwer rouge-score bert-score sacrebleu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 128.6 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7


In [3]:
import pandas as pd
import torch
import nltk
from rouge_score import rouge_scorer
from bert_score import score as bert_score
from sacrebleu import corpus_bleu
from jiwer import wer

# Ensure required NLTK data is available
nltk.download("wordnet")

# Function to compute METEOR-like score (approximation)
def compute_meteor(reference, candidate):
    return 1 - wer(reference, candidate)  # Word Error Rate as an inverse similarity measure

# Example usage
reference_text = "The cat sat on the mat."
generated_text = "The cat is sitting on the mat."
meteor_result = compute_meteor(reference_text, generated_text)

print(f"✅ METEOR Score (Approximate): {meteor_result:.4f}")


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
✅ METEOR Score (Approximate): 0.6667


In [4]:
df = pd.read_csv("/kaggle/input/inputi/summarization_results_all_scores (1).csv")  # Update with correct dataset path
df.head()


,Article,Human Summary,id,Generated Summary,ROUGE-1,ROUGE-2,ROUGE-L,BLEU,BERTScore,MoverScore
0,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...,f001ec5c4704938247d27a44948eebb37ae98d01,the formal accession was marked with a ceremon...,0.292135,0.114943,0.247191,0.054540,0.252550,3.838814
1,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b...",230c522854991d053fe98a718b1defa077a8efef,a stray dog in Washington state has used up at...,0.453608,0.189474,0.391753,0.164406,0.294844,4.521214
2,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...,4495ba8f3a340d97a9df1476f8a35502bcce1f69,"""long live Zarif,"" crowds chanted as his car r...",0.150000,0.025641,0.125000,0.594604,0.014261,5.919871
3,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...,a38e72fed88684ec8d60dd5856282e999dc8c0ca,five americans who were exposed to Ebola in we...,0.365217,0.123894,0.243478,0.078637,0.227430,3.047870
4,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...,c27cf1b136cc270023de959e7ab24638021bc43f,a student has admitted to hanging a noose made...,0.357143,0.121951,0.214286,0.142753,0.248586,4.163770


In [5]:
def compute_rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    scores = scorer.score(reference, candidate)
    return (
        scores["rouge1"].precision, scores["rouge1"].recall, scores["rouge1"].fmeasure,
        scores["rouge2"].precision, scores["rouge2"].recall, scores["rouge2"].fmeasure,
        scores["rougeL"].precision, scores["rougeL"].recall, scores["rougeL"].fmeasure
    )

df[[
    "ROUGE-1 Precision", "ROUGE-1 Recall", "ROUGE-1 F1",
    "ROUGE-2 Precision", "ROUGE-2 Recall", "ROUGE-2 F1",
    "ROUGE-L Precision", "ROUGE-L Recall", "ROUGE-L F1"
]] = df.apply(lambda x: compute_rouge(x["Human Summary"], x["Generated Summary"]), axis=1, result_type="expand")

print("✅ ROUGE Scores Computed")
df[["ROUGE-1 F1", "ROUGE-2 F1", "ROUGE-L F1"]].head()


✅ ROUGE Scores Computed


,ROUGE-1 F1,ROUGE-2 F1,ROUGE-L F1
0,0.292135,0.114943,0.247191
1,0.453608,0.189474,0.391753
2,0.150000,0.025641,0.125000
3,0.365217,0.123894,0.243478
4,0.357143,0.121951,0.214286


In [6]:
# Compute BERTScore using CPU
bert_p, bert_r, bert_f1 = bert_score(
    df["Generated Summary"].tolist(),
    df["Human Summary"].tolist(),
    lang="en",
    device="cpu"  # Force CPU usage
)

df["BERT Precision"] = bert_p.numpy()
df["BERT Recall"] = bert_r.numpy()
df["BERT F1"] = bert_f1.numpy()

print("✅ BERT Scores Computed")
df[["BERT Precision", "BERT Recall", "BERT F1"]].head()


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ BERT Scores Computed


,BERT Precision,BERT Recall,BERT F1
0,0.860706,0.887401,0.873850
1,0.879417,0.882565,0.880988
2,0.830892,0.836391,0.833633
3,0.859780,0.879668,0.869610
4,0.868025,0.878398,0.873181


In [7]:
df["BLEU Score"] = df.apply(lambda x: corpus_bleu([x["Generated Summary"]], [[x["Human Summary"]]]).score, axis=1)

print("✅ BLEU Scores Computed")
df[["BLEU Score"]].head()


✅ BLEU Scores Computed


,BLEU Score
0,5.266039
1,17.989570
2,0.979556
3,7.756734
4,5.951073


In [8]:
df.to_csv("/kaggle/working/summarization_scores.csv", index=False)
print("✅ Results saved as summarization_scores.csv")


✅ Results saved as summarization_scores.csv


In [9]:
df["METEOR Score"] = df.apply(lambda x: meteor_score([x["Human Summary"]], x["Generated Summary"]), axis=1)

print("✅ METEOR Scores Computed")
df[["METEOR Score"]].head()


NameError: name 'meteor_score' is not defined